In [1]:
import pandas as pd
import numpy as np
import os
from glob import glob
import lightgbm as lgb
from tqdm.notebook import tqdm
# from scipy import stats

In [2]:
data_dir = '/Volumes/Extreme SSD/rematch_eia_ferc1_docker'

dir_x_model_a = os.path.join(data_dir, 'working_data/model_a/model_a_x')
dir_x_model_b = os.path.join(data_dir, 'working_data/model_b/model_b_x')

dir_y_fit_model_a_ann = os.path.join(data_dir, 'working_data/model_a/model_a_ann_y_fit')
dir_y_fit_model_a_gbm = os.path.join(data_dir, 'working_data/model_a/model_a_gbm_y_fit')

dir_y_fit_model_b_ann = os.path.join(data_dir, 'working_data/model_b/model_b_ann_y_fit')
dir_y_fit_model_b_gbm = os.path.join(data_dir, 'working_data/model_b/model_b_gbm_y_fit')

dir_tranches = os.path.join(data_dir, 'working_data/tranches_ferc_to_eia')

dir_y_fit_out = os.path.join(data_dir, 'working_data/model_second_stage/model_z_gbm_y_fit')

fn_model2 = os.path.join(data_dir, 'working_data/model_second_stage/model_second_stage_training/model_2.txt')

fn_top_mappings = os.path.join(data_dir, 'output_data/top_mappings.parquet')

In [3]:
def extract_tranche_id(dir, colname):
    # For any given directory, search for all the applicable parquet files, and
    # return the results as a table with two columns, the parquet files (called whatever
    # you input as colname), and the extracted tranche_id
    # dir = dir_model_a
    ff = glob(dir + '/*.parquet')
    Cte = pd.DataFrame({colname:ff})
    Cte['tranche_id'] = Cte[colname].str.extract('([0-9]{4}_[0-9]{3}(?=\\.parquet))')
    Cte = Cte.set_index('tranche_id', drop=True)
    return Cte

def add_grouped_rank(ID, YFit):
    # For any ID file and YFit file, return a table with y_fit and the ranks for the y_fits
    Cte = ID[['record_id_ferc1']].copy()
    Cte = pd.concat([Cte, YFit], axis=1)
    Cte['y_fit_rank'] = Cte.groupby('record_id_ferc1')['y_fit'].rank(method='dense', ascending=False)
    Cte = Cte[['y_fit', 'y_fit_rank']]
    return Cte

In [4]:
mod2 = lgb.Booster(model_file=fn_model2)

In [5]:
FN = pd.concat([
        extract_tranche_id(dir=dir_tranches, colname='fn_id'),
        extract_tranche_id(dir=dir_x_model_a, colname='fn_x_a'),
        extract_tranche_id(dir=dir_x_model_b, colname='fn_x_b'),
        extract_tranche_id(dir=dir_y_fit_model_a_ann, colname='fn_y_fit_a_ann'),
        extract_tranche_id(dir=dir_y_fit_model_a_gbm, colname='fn_y_fit_a_gbm'),
        extract_tranche_id(dir=dir_y_fit_model_b_ann, colname='fn_y_fit_b_ann'),
        extract_tranche_id(dir=dir_y_fit_model_b_gbm, colname='fn_y_fit_b_gbm')
    ], axis=1, join="outer")

# add location for output
FN['y_fit_out'] = [os.path.join(dir_y_fit_out, 'model_z_gbm_y_fit__' + tranche_id + '.parquet') for tranche_id in FN.index.values]

FN.head(2)

fn_id  \
tranche_id                                                      
2001_000    /Volumes/Extreme SSD/rematch_eia_ferc1_docker/...   
2001_001    /Volumes/Extreme SSD/rematch_eia_ferc1_docker/...   

                                                       fn_x_a  \
tranche_id                                                      
2001_000    /Volumes/Extreme SSD/rematch_eia_ferc1_docker/...   
2001_001    /Volumes/Extreme SSD/rematch_eia_ferc1_docker/...   

                                                       fn_x_b  \
tranche_id                                                      
2001_000    /Volumes/Extreme SSD/rematch_eia_ferc1_docker/...   
2001_001    /Volumes/Extreme SSD/rematch_eia_ferc1_docker/...   

                                               fn_y_fit_a_ann  \
tranche_id                                                      
2001_000    /Volumes/Extreme SSD/rematch_eia_ferc1_docker/...   
2001_001    /Volumes/Extreme SSD/rematch_eia_ferc1_docker/...   

                                               fn_y_fit_a_gbm  \
tranche_id                                                      
2001_000    /Volumes/Extreme SSD/rematch_eia_ferc1_docker/...   
2001_001    /Volumes/Extreme SSD/rematch_eia_ferc1_docker/...   

                                               fn_y_fit_b_ann  \
tranche_id                                                      
2001_000    /Volumes/Extreme SSD/rematch_eia_ferc1_docker/...   
2001_001    /Volumes/Extreme SSD/rematch_eia_ferc1_docker/...   

                                               fn_y_fit_b_gbm  \
tranche_id                                                      
2001_000    /Volumes/Extreme SSD/rematch_eia_ferc1_docker/...   
2001_001    /Volumes/Extreme SSD/rematch_eia_ferc1_docker/...   

                                                    y_fit_out  
tranche_id                                                     
2001_000    /Volumes/Extreme SSD/rematch_eia_ferc1_docker/...  
2001_001    /Volumes/Extreme SSD/rematch_eia_ferc1_docker/...

The input for this model should look like this:
- X encoding A
- X encoding B
- y-fit, y-fit ranks from ANN A
- y-fit, y-fit ranks from GBM A
- y-fit, y-fit ranks from ANN B
- y-fit, y-fit ranks from GBM B

# Iterate

In [9]:
row = FN.iloc[1]

In [14]:
ID = pd.read_parquet(row['fn_id'])
X1A = pd.read_parquet(row['fn_x_a'])
X1B = pd.read_parquet(row['fn_x_b'])
YFit1AAnn = pd.read_parquet(row['fn_y_fit_a_ann'])
YFit1AGbm = pd.read_parquet(row['fn_y_fit_a_gbm'])
YFit1BAnn = pd.read_parquet(row['fn_y_fit_b_ann'])
YFit1BGbm = pd.read_parquet(row['fn_y_fit_b_gbm'])

X = np.hstack((
    X1A.values, 
    X1B.values,
    add_grouped_rank(ID=ID, YFit=YFit1AAnn).values,
    add_grouped_rank(ID=ID, YFit=YFit1AGbm).values,
    add_grouped_rank(ID=ID, YFit=YFit1BAnn).values,
    add_grouped_rank(ID=ID, YFit=YFit1BGbm).values
))

y_fit2 = mod2.predict(X)

ID = ID
Cte = ID.copy()
Cte['y_fit'] = y_fit2

TopMapping = Cte.sort_values(['record_id_ferc1', 'y_fit'], ascending=False).groupby('record_id_ferc1').head(5).reset_index(drop=True)

In [17]:
min(y_fit2)

-0.0801642323952331

In [21]:
Cte.sort_values(['record_id_ferc1', 'y_fit'], ascending=False).groupby('record_id_ferc1').head(5).reset_index(drop=True)

record_id_ferc1                               record_id_eia  \
0    f1_steam_2001_12_10_2_5   3118_ic_2001_plant_prime_mover_owned_1167   
1    f1_steam_2001_12_10_2_5  3118_ic_2001_plant_prime_mover_owned_15270   
2    f1_steam_2001_12_10_2_5    3118_ic_2001_plant_prime_mover_owned_963   
3    f1_steam_2001_12_10_2_5   3118_st_2001_plant_prime_mover_owned_1167   
4    f1_steam_2001_12_10_2_5   3118_ic_2001_plant_prime_mover_total_1167   
..                       ...                                         ...   
95  f1_steam_2001_12_108_1_2  2326_gt_2001_plant_prime_mover_total_13407   
96  f1_steam_2001_12_108_1_2                55182_2001_plant_owned_34407   
97  f1_steam_2001_12_108_1_2                55182_2001_plant_total_34491   
98  f1_steam_2001_12_108_1_2                55182_2001_plant_total_34407   
99  f1_steam_2001_12_108_1_2       55182_x718_2001_plant_gen_total_34407   

       y_fit  
0   0.220522  
1   0.159155  
2   0.113527  
3   0.073850  
4   0.068012  
..       ...  
95  0.172063  
96  0.071891  
97  0.069489  
98  0.069020  
99  0.067761  

[100 rows x 3 columns]

In [ ]:
is_top_mapping = Cte['y_fit_rank'] <= 5
TopMapping = Cte.loc[is_top_mapping].reset_index(drop=True)
top_mapping_dict[index] = TopMapping

OutputYFitRank.to_parquet(path=row['y_fit_out'], index=False)

In [22]:
top_mapping_dict = {tranche:None for tranche in FN.index.values}

for index, row in tqdm( FN.iterrows(), total=len(FN) ):
    
    ID = pd.read_parquet(row['fn_id'])
    X1A = pd.read_parquet(row['fn_x_a'])
    X1B = pd.read_parquet(row['fn_x_b'])
    YFit1AAnn = pd.read_parquet(row['fn_y_fit_a_ann'])
    YFit1AGbm = pd.read_parquet(row['fn_y_fit_a_gbm'])
    YFit1BAnn = pd.read_parquet(row['fn_y_fit_b_ann'])
    YFit1BGbm = pd.read_parquet(row['fn_y_fit_b_gbm'])
    
    X = np.hstack((
        X1A.values, 
        X1B.values,
        add_grouped_rank(ID=ID, YFit=YFit1AAnn).values,
        add_grouped_rank(ID=ID, YFit=YFit1AGbm).values,
        add_grouped_rank(ID=ID, YFit=YFit1BAnn).values,
        add_grouped_rank(ID=ID, YFit=YFit1BGbm).values
    ))
    
    y_fit2 = mod2.predict(X)
    
    ID = ID
    Cte = ID.copy()
    Cte['y_fit'] = y_fit2
    Cte['y_fit_rank'] = Cte.groupby('record_id_ferc1')['y_fit'].rank(method='dense', ascending=False)
    OutputYFitRank = Cte[['y_fit', 'y_fit_rank']]
    
    TopMappings = Cte.sort_values(['record_id_ferc1', 'y_fit'], ascending=False).groupby('record_id_ferc1').head(5).reset_index(drop=True)    
    top_mapping_dict[index] = TopMappings
    OutputYFitRank.to_parquet(path=row['y_fit_out'], index=False)

  0%|          | 0/1128 [00:00<?, ?it/s]

In [47]:
TopMappings = pd.concat(top_mapping_dict.values(), axis=0).reset_index(drop=True)

is_number_one = TopMappings['y_fit_rank'] == 1.0

TiesPerFerc = pd.DataFrame(TopMappings.loc[is_number_one]['record_id_ferc1'].value_counts())
TiesPerFerc = TiesPerFerc.rename(columns={'count':'num_matches'})

TopMappings = TopMappings.set_index('record_id_ferc1').merge(TiesPerFerc, left_index=True, right_index=True)
TopMappings['y_fit_rank'] = TopMappings['y_fit_rank'].astype(int)
# del TopMappings['num_matches']
TopMappings.to_parquet(fn_top_mappings, index=True)

In [ ]:
TopMappings = TopMappings.set_index('record_id_ferc1').merge(TiesPerFerc, left_index=True, right_index=True)
TopMappings['is_there_a_tie'] = TopMappings['num_matches'] >= 1
TopMappings['y_fit_rank'] = TopMappings['y_fit_rank'].astype(int)
del TopMappings['num_matches']
TopMappings.to_parquet(fn_top_mappings, index=True)

I've saved the top five suggested mappings, but that's mostly interesting for diagnostics-- IRL we'll just need a single mapping for each record_id_ferc1